# SI 330 Final Project - Brian Halperin

# 2016 Presidential Candidate Tweet Sentiment 


I will be comparing the sentiment analyis of Donald Trump tweets and Hilary Clinton tweets before and after the election. I am using a dataset which I downloaded from Kaggle.com which is a CSV file of both Hillary Clinton and Donald Trump tweets from January 5th, 2016 to September 28th 2016. In order to get their tweets from after September 28th and after November 8th, I will be using the Twitter API and Tweepy in order to easily scrape the tweets into a CSV file.

I wanted to compare the sentiment between Hillary Clinton and Donald Trump because I thought it would be an interesting metric to look at when comparing the two. I want to find out who has changed more since the election and who was more negative or positive going into the election. My expectations are that Donald's tweets will be more negative than Hillary's.

In [156]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "7919272-3x2-700x467.jpg")

In [26]:
import pandas as pd 
from pandas import DataFrame
import numpy as np
import requests
import json
import csv

Lets get our first dataset of tweets. The file is called "tweets.csv" and is 5.2 MB

In [4]:
both_tweets = pd.read_csv('tweets.csv') 
both_tweets.head(1) #Show one entry

,id,handle,text,is_retweet,original_author,time,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,is_quote_status,...,place_type,place_country_code,place_country,place_contained_within,place_attributes,place_bounding_box,source_url,truncated,entities,extended_entities
0,780925634159796224,HillaryClinton,The question in this election: Who can put the...,False,NaN,2016-09-28T00:22:34,NaN,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,https://studio.twitter.com,False,{'media': [{'display_url': 'pic.twitter.com/Xr...,{'media': [{'display_url': 'pic.twitter.com/Xr...


Great. Now, lets filter down our columns so we don't have to deal with all of these extra columns.

In [8]:
both_tweets = DataFrame(both_tweets,columns = ['handle','text','time'])
both_tweets.head(1) 

,handle,text,time
0,HillaryClinton,The question in this election: Who can put the...,2016-09-28T00:22:34


Let's do some data manipulation on the time column so that we can just get the date without the time.

In [9]:
def fix_time(time): #Define a function to strip last 10 characters.
    time = time[0:10]
    return time

In [13]:
both_tweets['time'] = both_tweets['time'].apply(fix_time) #Apply function to time column.
both_tweets.head(6)

,handle,text,time
0,HillaryClinton,The question in this election: Who can put the...,2016-09-28
1,HillaryClinton,"Last night, Donald Trump said not paying taxes...",2016-09-27
2,HillaryClinton,Couldn't be more proud of @HillaryClinton. Her...,2016-09-27
3,HillaryClinton,"If we stand together, there's nothing we can't...",2016-09-27
4,HillaryClinton,Both candidates were asked about how they'd co...,2016-09-27
5,realDonaldTrump,Join me for a 3pm rally - tomorrow at the Mid-...,2016-09-27


In order to do sentiment analyiss on the two candidates, we need to seperate their tweets from eachother. We can do this by creating two new dataframes with only Donald and only Hillary tweets.

In [14]:
pre_hillary = both_tweets[both_tweets['handle']=='HillaryClinton']
pre_donald = both_tweets[both_tweets['handle']=='realDonaldTrump']

In [16]:
pre_donald.head(2)

,handle,text,time
5,realDonaldTrump,Join me for a 3pm rally - tomorrow at the Mid-...,2016-09-27
8,realDonaldTrump,"Once again, we will have a government of, by a...",2016-09-27


In [17]:
pre_hillary.head(2)

,handle,text,time
0,HillaryClinton,The question in this election: Who can put the...,2016-09-28
1,HillaryClinton,"Last night, Donald Trump said not paying taxes...",2016-09-27


Cool. Now we have all of Donald Trump's and Hillary Clinton's tweets from January 5th 2016 to September 5th 2016 in one dataframe. However, we are doing sentiment analyis based on tweets before and after the election so we need the tweets from September 5th and forward from both Donald Trump and Hillary Clinton. We can do this by using the twitter API and using a module called tweepy to easily make a CSV file based on their tweets. I've compiled this code in PyCharm as I found it difficult to pip install tweepy in the Jupyter Notebook.


First, lets import tweepy and our CSV module. Next, lets make variables for our API information.

In [ ]:
import tweepy
import csv

#Twitter API Information. Attained my making an account.
consumer_key = "jz2rnCUkEoxMwOdSotlVvkdPC"
consumer_secret = "60Uwmgm6OAXgeryyNFbnBy815CRun3iatJtvy2JWDUcYj4R5h0"
access_key = "1363213442-AfhSEqOMy6L6F8qJwuS9sdejqrTaCCL1KOzUpYS"
access_secret = "8mXke5lhyUrFyRvBm9ZHVhhELQN6Ru345da8MnUnhv0E2"

Now let's define a function that will get the most recent tweets from a given handle and write a CSV file with three fields, the tweet ID, the time of the tweet, and the content of the tweet.

In [ ]:
#Get as many tweets as possible from the handle.
def get_tweets(handle): 
    #Log into twitter API via information, start tweepy module.
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)
  
    tweets = [] #Create list for tweets 
    new_tweets = api.user_timeline(screen_name = handle,count=200) #Make request for tweets. 
    tweets.extend(new_tweets) #Save tweets
    oldest = tweets[-1].id - 1 #save id of the oldest tweet in order to prevent getting same tweets over and over.

    while len(new_tweets) > 0: #Keep getting tweets
        new_tweets = api.user_timeline(handle = handle,count=200,max_id=oldest)
        tweets.extend(new_tweets) #save most recent tweets
        oldest = tweets[-1].id - 1
    
    for tweet in tweets:    #Create Tweepy object for easy CSV entry
        outtweets = [[tweet.id_str, tweet.created_at, tweet.text.encode('utf-8')]]
        
    with open('%s_tweets.csv' % handle, 'w') as f: #Save as a CSV with file name as the 'handle_tweets.csv'
        writer = csv.writer(f)
        writer.writerow(["id","created_at","text"])
        writer.writerows(outtweets)
    pass

Great. Now let's call the function on Hilary's and Donald's handles.

In [ ]:
get_tweets('Hillary_Clinton')
get_tweets("realDonaldTrump")

Now, we have a list of tweets that was scrapped using Twitter's API and the Tweepy Module. Let's use pandas in order to load these CSV's into the Jupyter Notebook. The two datasets we downloaded are called HillaryClinton_tweets.csv and realDonaldTrump tweets and are 582kb, and 480kb respectfully. 

In [71]:
hil_scrape = pd.read_csv("HillaryClinton_tweets.csv")
don_scrape = pd.read_csv("realDonaldTrump_tweets.csv")

In [72]:
hil_scrape.head(2)

,id,created_at,text
0,802721447026098176,2016-11-27 03:51:21,b'RT @marceelias: I just published \xe2\x80\x9...
1,801888191276089344,2016-11-24 20:40:17,b'I was greeted by this heartwarming display o...


In [73]:
don_scrape.head(2)

,id,created_at,text
0,806214236053667842,2016-12-06 19:10:27,b'Masa said he would never do this had we (Tru...
1,806214078465245185,2016-12-06 19:09:49,b'Masa (SoftBank) of Japan has agreed to inves...


It looks like there is some data manipulation we need to do in order to fix our text. We need to remove the b' that is coming before every tweet. Let's also use our Fix_Time function to reformat the time. 

In [74]:
def fix_text(text):
    text = text[2:]
    return text
don_scrape['text']=don_scrape['text'].apply(fix_text)
don_scrape['created_at'] = don_scrape['created_at'].apply(fix_time)
hil_scrape['text']=hil_scrape['text'].apply(fix_text)
hil_scrape['created_at'] = hil_scrape['created_at'].apply(fix_time)


In [75]:
hil_scrape.head(3)

,id,created_at,text
0,802721447026098176,2016-11-27,RT @marceelias: I just published \xe2\x80\x9cL...
1,801888191276089344,2016-11-24,I was greeted by this heartwarming display on ...
2,801887362787770368,2016-11-24,Hoping everyone has a safe &amp; Happy Thanksg...


In [76]:
don_scrape.head(3)

,id,created_at,text
0,806214236053667842,2016-12-06,Masa said he would never do this had we (Trump...
1,806214078465245185,2016-12-06,Masa (SoftBank) of Japan has agreed to invest ...
2,806177746397306883,2016-12-06,"Join me tonight in Fayetteville, North Carolin..."


We are going to merge our two datasets together now. This requires a little bit of data manipulation in the form of making sure the tables each have the same columns. We are going to delete the ID column in the scrape table and add a handle column with the handle name. We are also going to change the name of the "created_at" column to "time" in order to match the other dataframe.

In [79]:
hil_scrape.rename(columns = {'created_at':'time'}, inplace=True)
don_scrape.rename(columns = {'created_at':'time'}, inplace=True)
hil_scrape['handle'] = 'HillaryClinton'
don_scrape['handle'] = 'realDonaldTrump'



In [80]:
hil_scrape = hil_scrape.drop('id',axis=1)
don_scrape = don_scrape.drop('id',axis = 1)

In [82]:
hil_scrape.head(2)

,time,text,handle
0,2016-11-27,RT @marceelias: I just published \xe2\x80\x9cL...,HillaryClinton
1,2016-11-24,I was greeted by this heartwarming display on ...,HillaryClinton


In [87]:
don_scrape.head(2)

,time,text,handle
0,2016-12-06,Masa said he would never do this had we (Trump...,realDonaldTrump
1,2016-12-06,Masa (SoftBank) of Japan has agreed to invest ...,realDonaldTrump


Now that both dataframes have the same columns, we can merge them in order to get a complete list of tweets from January 5th 2016 to the present. Because we are getting the tweets through the Twitter API, we are getting way more tweets than we actually need in our scraped dataset. This will lead to duplicate values. We need to make our dataframe smaller so we are not making a dataframe with duplicate tweets. If we take a look at our datasets, we can see that the last tweet from our pre_election data for both Donald and Hillary, we can find the number of tweets we want from the dataset. We want 1417 tweets from Hillary from our scraped data, and we want 785 tweets from Donald from our scraped data.


In [88]:
don_scrape = don_scrape.head(785)
hil_scrape = hil_scrape.head(1417)

In [99]:
hil_merge = pd.concat([hil_scrape,pre_hillary], join = 'inner')
don_merge = pd.concat([don_scrape,pre_donald], join ='inner')
don_merge.head(3)

,handle,text,time
0,realDonaldTrump,Masa said he would never do this had we (Trump...,2016-12-06
1,realDonaldTrump,Masa (SoftBank) of Japan has agreed to invest ...,2016-12-06
2,realDonaldTrump,"Join me tonight in Fayetteville, North Carolin...",2016-12-06


In [100]:
don_merge.tail(3)

,handle,text,time
6441,realDonaldTrump,"""@ghosthunter_lol: Iowa key endorsement for @r...",2016-01-05
6442,realDonaldTrump,"""@iLoveiDevices: @EdwinRo47796972 @happyjack22...",2016-01-05
6443,realDonaldTrump,"""@SalRiccobono: @realDonaldTrump @troyconway D...",2016-01-05


Great! Now we have complete dataframes of both of tweets from January 5th 2016 to December 2016. Our next step will be to split them up from pre-election and post-election and then do our sentiment analysis based on those time periods. But first, let's convert these dates into Pandas datetime in order to make it easier


In [101]:
import datetime
don_merge['time'] = pd.to_datetime(don_merge['time'])
hil_merge['time'] = pd.to_datetime(hil_merge['time'])

In [102]:
don_merge.head(2)

,handle,text,time
0,realDonaldTrump,Masa said he would never do this had we (Trump...,2016-12-06
1,realDonaldTrump,Masa (SoftBank) of Japan has agreed to invest ...,2016-12-06


In [108]:
don_merge.tail(2)

,handle,text,time
6442,realDonaldTrump,"""@iLoveiDevices: @EdwinRo47796972 @happyjack22...",2016-01-05
6443,realDonaldTrump,"""@SalRiccobono: @realDonaldTrump @troyconway D...",2016-01-05


If we look at our Donald dataframe, we can see that we should  seperate the first 122 entries from the last 3881 in order to get a complete dataframe of pre-election tweets and post-election tweets. 

If we look at our Hillary dataframe, we can see that we should seperate the first 29 entries from the last 4614 entries in order to get a complete dataframe of pre-election and post-election tweets.

In [132]:
pre_election_don = don_merge.tail(3881)
post_election_don = don_merge.head(122)

pre_election_hil = hil_merge.tail(4614)
post_eleciton_hil = hil_merge.head(29)

Now that we have our four dataframes, two for Hillary and two for Donald, we can start to do our sentiment analysis! For our sentiment anaylis, I am using the machine learning API from the following website: 

http://www.datumbox.com/machine-learning-api/

I will send a request with text to the API and will receive a JSON response with the sentiment of the request. First, let's define a function to make a call to the API. The documentation for the API can be found here:

http://www.datumbox.com/files/API-Documentation-1.0v.pdf

The reason for the if statement is because we want to first check if status is set to 1 before trying to access the result key, because if status is set to 0 there won't be a key:

In [179]:
def call(text):
    api_key = '1645ffbd01ce56be8d448fe8cd6a7ce2'
    base_url = "http://api.datumbox.com/1.0/TwitterSentimentAnalysis.json?"
    response = requests.get(base_url+'api_key='+api_key+'&'+'text='+text)
    data = response.json()
    if data['output']['status']:
        return data['output']['result']
    else:
        raise SomeException('No result, error {0[ErrorCode]}, {0[ErrorMessage]!r}'.format(
            data['output']['error']))

So now we have a function that we can use to find the sentiment of our tweets. Now, let's make some dictionaries so we can get a count of how many tweets are positive, neutral or negative.

In [158]:
pre_hil_dic = {}
pre_hil_dic['positive'] = 0
pre_hil_dic['negative'] = 0
pre_hil_dic['neutral'] = 0

pre_trump_dic = {}
pre_trump_dic['positive'] = 0
pre_trump_dic['negative'] = 0
pre_trump_dic['neutral'] = 0

post_hil_dic = {}
post_hil_dic['positive'] = 0
post_hil_dic['negative'] = 0
post_hil_dic['neutral'] = 0

post_trump_dic = {}
post_trump_dic['positive'] = 0
post_trump_dic['negative'] = 0
post_trump_dic['neutral'] = 0

Now that we have our dictionaries, we can iterate through our tweets and get a count of of the sentiment. Because we are limited to 1000 calls per day of the API, we cannot do sentiment anaylsis for all the tweets as planned. Instead, we will only iterate through 350 of each Donald and Hillary's pre-election tweets, and the remainder of their post-election tweets. I chose the number 350 because it will allow us to still call the API after the iterations, if needed. 

In [164]:
count = 0
for tweet in pre_election_hil['text'].head(350):
    count = count + 1
    if count % 25 == 0:
        print(count)
    if call(tweet) == 'neutral':
        pre_hil_dic['neutral'] = pre_hil_dic['neutral'] + 1
    elif call(tweet) == 'positive':
        pre_hil_dic['positive'] = pre_hil_dic['positive'] + 1
    elif call(tweet) == 'negative':
        pre_hil_dic['negative'] = pre_hil_dic['negative'] + 1


25
50
75
100
125
150
175
200
225
250
275
300
325
350


In [183]:
count = 0
for tweet in pre_election_don['text'].head(350):
    count = count + 1
    if count % 25 == 0:
        print(count)
    if call(tweet) == 'neutral':
        pre_trump_dic['neutral'] = pre_trump_dic['neutral'] + 1
    elif call(tweet) == 'positive':
        pre_trump_dic['positive'] = pre_trump_dic['positive'] + 1
    elif call(tweet) == 'negative':
        pre_trump_dic['negative'] = pre_trump_dic['negative'] + 1



25
50
75
100
125
150
175
200
225
250
275
300
325
350


In [184]:
count = 0
for tweet in post_election_don['text']:
    count = count + 1
    if count % 25 == 0:
        print(count)
    if call(tweet) == 'neutral':
        post_trump_dic['neutral'] = post_trump_dic['neutral'] + 1
    elif call(tweet) == 'positive':
        post_trump_dic['positive'] = post_trump_dic['positive'] + 1
    elif call(tweet) == 'negative':
        post_trump_dic['negative'] = post_trump_dic['negative'] + 1


25
50
75
100


In [192]:
count = 0
for tweet in post_eleciton_hil:
    count = count + 1
    if call(tweet) == 'neutral':
        post_hil_dic['neutral'] = post_hil_dic['neutral'] + 1
    elif call(tweet) == 'positive':
        post_hil_dic['positive'] = post_hil_dic['positive'] + 1
    elif call(tweet) == 'negative':
        post_hil_dic['negative'] = post_hil_dic['negative'] + 1


Great. Not that we have iterated through each of our dataframes, we can look at the results.


In [169]:
pre_trump_dic

{'negative': 37, 'neutral': 142, 'positive': 118}

In [186]:
post_trump_dic

{'negative': 21, 'neutral': 53, 'positive': 48}

In [187]:
post_hil_dic

{'negative': 1, 'neutral': 0, 'positive': 2}

In [188]:
pre_hil_dic

{'negative': 31, 'neutral': 264, 'positive': 144}

Great. Now let's take a look at our results

## Pre_Trump_Dic:

37/297 = 12.45% of tweets are negative. 

142/297 = 47.81% of tweets are neutral.

118/297 = 39.73% of tweets are positive.

## Post_Trump_Dic: 

21/122 = 17.21% of tweets are negative

53/122 = 42.44% of tweets are neutral 

48/122 = 39.34% of tweets are positive

## Pre_Hil_Dic:

31/439 = 7.63% of tweets are negative 

264/439 = 60% of tweets are neutral 

144/439 = 32.36% of tweets are positive

## Post_Hil_Dic:

1/3 = 33% of tweets are negative 

0/3 = 0% of tweets are neutral

2/3 = 66% of tweets are positive


Very interesting. However, we can make this easier to look at by making pie charts for anaylsis. Because of time, I won't be using a module like Seaborn, instead I will be using https://www.meta-chart.com/ which is a cool website that allows you to create pie charts. A pie chart is the best choice for our data because we have percentages.

In [200]:
Image(url= "pre_don.png")

In [201]:
Image(url= "post_don.png")

Wow, very intresting. Since the election, Donald has gotten more negative and less positive. He has stayed as neutral as he was before. Let's take a look at Hillary's graphs

In [211]:
Image(url= "prehil.jpg")

In [209]:
Image(url= "post_hil.png")

Great. Because Hillary has only tweeted 21 times since the Election, our results are going to be a little off. We can see that since the election, Hillary has tweeted more positive messages.

## Conclusion

There is definitely a bunch of things I would have done differently with my project if I had more time. Firstly, one of my setbacks was that the datumbox API can only be used 1000 times a day. If I had more time, I would have used another method for sentiment analysis that could iterate through all the tweets at once. Another thing I would do differently if I had more time would be using the Seaborn module to create the pie charts. Lastly, if I could switch another aspect it would definitely be that I would wait another couple months to do my analysis. Because we are still pretty close to November 8th, Hillary has been extremely quiet on Twitter, unlike Donald. This sort of screwed up some of my results, because there were only three Hillary tweets that got a response from the API. Overall, I really enjoyed doing this project and some cool data analysis about the candidates.